In [1]:
# nuclio: ignore
# if the nuclio-jupyter package is not installed run !pip install nuclio-jupyter
import nuclio 

In [ ]:
%%nuclio cmd
pip install kfserving
pip install numpy
pip install xgboost

In [3]:
import kfserving
import os
import numpy as np
from typing import List, Any

import xgboost as xgb

In [4]:
BOOSTER_FILE = "model.xgboost"

class XGBoostModel(kfserving.KFModel):
    def __init__(self, name: str, model_dir: str, booster: xgb.XGBModel = None):
        super().__init__(name)
        self.name = name
        self.model_dir = model_dir
        if not booster is None:
            self._booster = booster
            self.ready = True

    def load(self):
        model_file = os.path.join(
            kfserving.Storage.download(self.model_dir), BOOSTER_FILE)
        self._booster = xgb.Booster(model_file=model_file)
        self.ready = True

    def predict(self, body: List) -> List:
        try:
            # Use of list as input is deprecated see https://github.com/dmlc/xgboost/pull/3970
            dmatrix = xgb.DMatrix(body)
            result: xgb.DMatrix = self._booster.predict(dmatrix)
            return result.tolist()
        except Exception as e:
            raise Exception("Failed to predict %s" % e)


In [5]:
# nuclio: end-code
from mlrun import new_model_server
import requests

In [6]:
fn = new_model_server('iris-xgboost-advanced', models={'iris_v2': '/User/models/iris_v2'}, model_class='XGBoostModel')
fn.with_v3io('User','~/') 
print(f'Function yaml (dict):\n{fn.to_dict()}')

Function yaml (dict):
{'kind': 'remote', 'metadata': {'name': 'iris-xgboost-advanced'}, 'spec': {'command': '', 'args': [], 'image': '', 'mode': '', 'description': '', 'volumes': [{'volume': {'flexVolume': {'driver': 'v3io/fuse', 'options': {'accessKey': '460ee949-eaaa-468a-ae26-5fdb9156996e', 'container': 'users', 'subPath': '/admin'}}, 'name': 'fs'}, 'volumeMount': {'name': 'fs', 'mountPath': 'User'}}], 'env': {'SERVING_MODEL_iris_v2': '/User/models/iris_v2', 'TRANSPORT_PROTOCOL': 'seldon', 'ENABLE_EXPLAINER': 'False', 'MODEL_CLASS': 'XGBoostModel', 'V3IO_USERNAME': 'admin', 'V3IO_ACCESS_KEY': '460ee949-eaaa-468a-ae26-5fdb9156996e', 'V3IO_API': 'v3io-webapi.default-tenant.svc:8081'}, 'config': {'spec.triggers.http': {'kind': 'http', 'maxWorkers': 8, 'attributes': {'ingresses': {}}, 'annotations': {}}}, 'build_commands': [], 'base_spec': {'apiVersion': 'nuclio.io/v1', 'kind': 'Function', 'metadata': {'annotations': {'nuclio.io/generated_by': 'function generated at 23-10-2019 by admin 

In [7]:
addr = fn.deploy(project='nuclio-serving')

[nuclio] 2019-10-23 07:45:38,702 (info) Building processor image
[nuclio] 2019-10-23 07:45:46,793 (info) Build complete
[nuclio] 2019-10-23 07:45:54,900 (info) Function deploy complete
[nuclio] 2019-10-23 07:45:54,906 done updating iris-xgboost-advanced, function address: 192.168.206.37:31940


In [8]:
import requests

# Get sample event 
samples = [[6.4, 2.8, 5.6, 2.2], [5.2, 3.4, 1.4, 0.2]]

# Seldon protocol event
event_seldon = {"data": {"ndarray": samples}}
event_general = {"data": samples}
even_tensorflow = {"instances": samples}
event = event_seldon

csel = str(event).replace("\'", "\"")
print(f'Sent event:\n{csel}\n')

resp = requests.put(addr + '/predict/iris_v2', data=csel)
print(f'Prediction (Response):\n{resp.content}')

Sent event:
{"data": {"ndarray": [[6.4, 2.8, 5.6, 2.2], [5.2, 3.4, 1.4, 0.2]]}}

Prediction (Response):
b'{"data": {"ndarray": [[0.01101686805486679, 0.013427456840872765, 0.9755556583404541], [0.973975419998169, 0.014415861107409, 0.011608744971454144]]}}'
